In [ ]:
!pip install tensorboardX
import os
import requests
import zipfile
from pathlib import Path
import torch
import torch.optim as optim
import time
import math
import matplotlib.pyplot as plt
from tqdm.notebook import tqdm  # Use the Notebook-friendly progress bar
from tensorboardX import SummaryWriter
from torch.optim.lr_scheduler import ReduceLROnPlateau
import glob
from sklearn.model_selection import train_test_split
from torch.utils.data import Dataset, DataLoader
from PIL import Image
import torchvision.transforms as transforms
from io import BytesIO
import torch.nn as nn
import numpy as np
import torchvision.models as models
from torchvision.models import VGG16_Weights
import random


device="cpu"
if torch.cuda.is_available():
    device = "cuda"
    print(f"✅ Successfully connected to GPU: {torch.cuda.get_device_name(0)}")

# 2. Define download paths
data_dir = Path('./data/coco')
data_dir.mkdir(parents=True, exist_ok=True)

# 3. Download COCO 2017 Validation Set (as our training material)
# This is a compromise: the full Train set is huge; the Val set has 5000 images and is enough for the Style Transfer demo
coco_url = "http://images.cocodataset.org/zips/val2017.zip"
zip_path = data_dir / "val2017.zip"

if not (data_dir / "val2017").exists():
    print("⬇️ Downloading the COCO dataset (~1GB), please wait...")
    !wget -q -P {data_dir} {coco_url}

    print("📦 Extracting...")
    with zipfile.ZipFile(zip_path, 'r') as zip_ref:
        zip_ref.extractall(data_dir)

    # Delete the zip to save space
    os.remove(zip_path)
    print("✅ Dataset is ready!")
else:
    print("✅ Dataset already exists, skipping download.")

In [ ]:
# 1. Collect all image paths
image_dir = data_dir / "val2017"
all_image_paths = sorted(list(image_dir.glob("*.jpg")))
print(f"Total number of images: {len(all_image_paths)}")

# 2. Split the dataset
# First separate a test set (e.g., reserve 50 images for the final demo)
train_val_paths, test_paths = train_test_split(all_image_paths, test_size=50, random_state=42)

# Then split the rest into Train and Validation (e.g., 9:1)
train_paths, val_paths = train_test_split(train_val_paths, test_size=0.1, random_state=42)

print(f"--- Dataset split details (record these numbers in the report) ---")
print(f"🔹 Training set (for training): {len(train_paths)} images")
print(f"🔸 Validation set (for plotting the loss curve): {len(val_paths)} images")
print(f"🚀 Test set (for the final showcase): {len(test_paths)} images")


# 3. Define the PyTorch Dataset class
class ContentDataset(Dataset):
    def __init__(self, image_paths, transform=None):
        self.image_paths = image_paths
        self.transform = transform

    def __len__(self):
        return len(self.image_paths)

    def __getitem__(self, idx):
        img_path = self.image_paths[idx]
        try:
            image = Image.open(img_path).convert("RGB")
            if self.transform:
                image = self.transform(image)
            return image
        except Exception as e:
            print(f"Error loading {img_path}: {e}")
            return torch.zeros(3, 1, 1)  # Return an all-black image to avoid crashes


# 4. Define preprocessing (Transforms)
# Style transfer usually resizes images to a fixed size (e.g., 256x256) and normalizes them
class AddGaussianNoise(object):
    def __init__(self, mean=0.0, std=0.05):
        self.mean = mean
        self.std = std

    def __call__(self, tensor):
        # tensor is a [C, H, W] tensor and stays within 0 to 1
        noise = torch.randn_like(tensor) * self.std + self.mean
        noised = tensor + noise
        # Keep it within the 0 to 1 range
        return torch.clamp(noised, 0.0, 1.0)

    def __repr__(self):
        return f"{self.__class__.__name__}(mean={self.mean}, std={self.std})"

train_transform = transforms.Compose([
    transforms.Resize(256),
    transforms.CenterCrop(256),
    transforms.ToTensor(),
    AddGaussianNoise(mean=0.0, std=0.05),
])

# 5. Create DataLoaders
batch_size = 4  # Adjust based on Colab memory; if OOM, change to 2 or 1

train_dataset = ContentDataset(train_paths, transform=train_transform)
val_dataset = ContentDataset(val_paths, transform=train_transform)
# Test dataset usually doesn't need a DataLoader; just read single images for display

train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True, num_workers=2)
val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False, num_workers=2)

print("✅ DataLoaders are ready!")

In [ ]:
# 1. Define the download function
def load_image_from_url(url, target_size=None):
    # Pretend to be a browser to avoid being blocked
    headers = {
        'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.3'
    }
    try:
        response = requests.get(url, headers=headers, timeout=10)
        response.raise_for_status()  # Check for 403/404 errors

        img = Image.open(BytesIO(response.content)).convert('RGB')

        if target_size:
            img = img.resize(target_size, Image.BICUBIC)
        return img

    except Exception as e:
        print(f"❌ Download failed: {e}")
        return None


# 2. Download 'Starry Night'
# Backup URL (if Wikimedia still fails, use this fallback)
style_url = "https://upload.wikimedia.org/wikipedia/commons/thumb/e/ea/Van_Gogh_-_Starry_Night_-_Google_Art_Project.jpg/1024px-Van_Gogh_-_Starry_Night_-_Google_Art_Project.jpg"
style_image_name = "starry_night.jpg"

print(f"⬇️ Downloading style image: {style_image_name}...")
style_img_pil = load_image_from_url(style_url, target_size=(256, 256))

if style_img_pil is not None:
    style_img_pil.save(style_image_name)

    # 3. Preprocess the style image
    style_transform = transforms.Compose([
        transforms.ToTensor(),
    ])

    # Add the batch dimension
    style_img_tensor = style_transform(style_img_pil).unsqueeze(0).to(device)
    print(f"✅ Style image ready, tensor shape: {style_img_tensor.shape}")

    # 4. Visualize to verify
    plt.figure(figsize=(5, 5))
    plt.imshow(style_img_pil)
    plt.title("Style Image: Starry Night")
    plt.axis('off')
    plt.show()
else:
    print("⚠️ Image download failed. Try another URL or manually upload 'starry_night.jpg' to the Colab file panel.")

In [ ]:
# ==========================================
# 1. Basic components
# ==========================================

class ConvLayer(nn.Module):
    def __init__(self, in_channels, out_channels, kernel_size, stride):
        super(ConvLayer, self).__init__()
        reflection_padding = kernel_size // 2
        self.reflection_pad = nn.ReflectionPad2d(reflection_padding)
        self.conv2d = nn.Conv2d(in_channels, out_channels, kernel_size, stride)

    def forward(self, x):
        out = self.reflection_pad(x)
        out = self.conv2d(out)
        return out


class LightweightConvLayer(nn.Module):
    def __init__(self, in_channels, out_channels, kernel_size, stride):
        super(LightweightConvLayer, self).__init__()
        reflection_padding = kernel_size // 2
        self.reflection_pad = nn.ReflectionPad2d(reflection_padding)

        # 1. Depthwise convolution: groups=in_channels, extracts spatial features
        # Followed by Norm, so bias=False
        self.depthwise = nn.Conv2d(in_channels, in_channels, kernel_size=kernel_size,
                                   stride=stride, groups=in_channels, bias=False)

        # [New] Insert Norm and Activation between them to increase nonlinearity
        self.dw_norm = nn.InstanceNorm2d(in_channels, affine=True)
        self.dw_relu = nn.ReLU(inplace=True)

        # 2. Pointwise convolution: 1x1 conv to combine channel features
        self.pointwise = nn.Conv2d(in_channels, out_channels, kernel_size=1,
                                   stride=1, bias=False)

    def forward(self, x):
        out = self.reflection_pad(x)

        # Depthwise stage
        out = self.depthwise(out)
        out = self.dw_norm(out)
        out = self.dw_relu(out)

        # Pointwise stage
        out = self.pointwise(out)

        return out


# ==========================================
# 2. Residual block
# ==========================================

class ResidualBlock(nn.Module):
    """Standard Residual Block"""
    def __init__(self, channels):
        super(ResidualBlock, self).__init__()
        self.conv1 = ConvLayer(channels, channels, kernel_size=3, stride=1)
        self.in1 = nn.InstanceNorm2d(channels, affine=True)
        self.conv2 = ConvLayer(channels, channels, kernel_size=3, stride=1)
        self.in2 = nn.InstanceNorm2d(channels, affine=True)
        self.relu = nn.ReLU()

    def forward(self, x):
        residual = x
        out = self.relu(self.in1(self.conv1(x)))
        out = self.in2(self.conv2(out))
        out = out + residual
        return out


class LightweightResidualBlock(nn.Module):
    """Lightweight Residual Block (Depthwise Separable)"""
    def __init__(self, channels):
        super(LightweightResidualBlock, self).__init__()
        # Replace the standard ConvLayer with LightweightConvLayer
        self.conv1 = LightweightConvLayer(channels, channels, kernel_size=3, stride=1)
        self.in1 = nn.InstanceNorm2d(channels, affine=True)
        self.conv2 = LightweightConvLayer(channels, channels, kernel_size=3, stride=1)
        self.in2 = nn.InstanceNorm2d(channels, affine=True)
        self.relu = nn.ReLU()

    def forward(self, x):
        residual = x
        out = self.relu(self.in1(self.conv1(x)))
        out = self.in2(self.conv2(out))
        out = out + residual
        return out


# ==========================================
# 3. Upsampling layer
# ==========================================

class UpsampleConvLayer(nn.Module):
    def __init__(self, in_channels, out_channels, kernel_size, stride, upsample=None):
        super(UpsampleConvLayer, self).__init__()
        self.upsample = upsample
        reflection_padding = kernel_size // 2
        self.reflection_pad = nn.ReflectionPad2d(reflection_padding)
        self.conv2d = nn.Conv2d(in_channels, out_channels, kernel_size, stride)

    def forward(self, x):
        x_in = x
        if self.upsample:
            x_in = torch.nn.functional.interpolate(x_in, mode='nearest', scale_factor=self.upsample)
        out = self.reflection_pad(x_in)
        out = self.conv2d(out)
        return out

class LightweightUpsampleConvLayer(nn.Module):
    """Lightweight upsampling layer: Upsample + Lightweight Conv"""
    def __init__(self, in_channels, out_channels, kernel_size, stride, upsample=None):
        super(LightweightUpsampleConvLayer, self).__init__()
        self.upsample = upsample
        # Reuse LightweightConvLayer
        self.conv = LightweightConvLayer(in_channels, out_channels, kernel_size, stride)
    def forward(self, x):
        x_in = x
        if self.upsample:
            x_in = torch.nn.functional.interpolate(x_in, mode='nearest', scale_factor=self.upsample)
        out = self.conv(x_in)
        return out

# ==========================================
# 4. Model definition
# ==========================================

class BaselineTransformerNet(nn.Module):
    def __init__(self):
        super(BaselineTransformerNet, self).__init__()
        # Initial convolution layers
        self.conv1 = ConvLayer(3, 32, kernel_size=9, stride=1)
        self.in1 = nn.InstanceNorm2d(32, affine=True)
        self.conv2 = ConvLayer(32, 64, kernel_size=3, stride=2)
        self.in2 = nn.InstanceNorm2d(64, affine=True)
        self.conv3 = ConvLayer(64, 128, kernel_size=3, stride=2)
        self.in3 = nn.InstanceNorm2d(128, affine=True)

        # Residual layers (Standard)
        self.res1 = ResidualBlock(128)
        self.res2 = ResidualBlock(128)
        self.res3 = ResidualBlock(128)
        self.res4 = ResidualBlock(128)
        self.res5 = ResidualBlock(128)

        # Upsampling Layers
        self.deconv1 = UpsampleConvLayer(128, 64, kernel_size=3, stride=1, upsample=2)
        self.in4 = nn.InstanceNorm2d(64, affine=True)
        self.deconv2 = UpsampleConvLayer(64, 32, kernel_size=3, stride=1, upsample=2)
        self.in5 = nn.InstanceNorm2d(32, affine=True)
        self.deconv3 = ConvLayer(32, 3, kernel_size=9, stride=1)

        self.relu = nn.ReLU()

    def forward(self, X):
        y = self.relu(self.in1(self.conv1(X)))
        y = self.relu(self.in2(self.conv2(y)))
        y = self.relu(self.in3(self.conv3(y)))
        y = self.res1(y)
        y = self.res2(y)
        y = self.res3(y)
        y = self.res4(y)
        y = self.res5(y)
        y = self.relu(self.in4(self.deconv1(y)))
        y = self.relu(self.in5(self.deconv2(y)))
        y = self.deconv3(y)
        return torch.sigmoid(y)

class LightweightTransformerNet(nn.Module):
    def __init__(self, mode='v1'):
        """
        Args:
            mode (str):
                'v1' - Only Residual Blocks use lightweight conv (Baseline Encoder/Decoder)
                'v2' - Encoder + Residual Blocks use lightweight conv
                'v3' - Fully lightweight (Encoder + Res + Decoder)
        """
        super(LightweightTransformerNet, self).__init__()
        self.mode = mode
        print(f"🏗️ Initializing Lightweight Model with mode: {mode}")

        # 1. Dynamically choose layer types (Dynamic Layer Selection)
        # ---------------------------------------------------
        # Encoder Layer Type
        if mode in ['v2', 'v3']:
            EncLayer = LightweightConvLayer
        else:
            EncLayer = ConvLayer  # v1 uses standard conv as the encoder

        # Decoder Layer Type
        if mode == 'v3':
            UpLayer = LightweightUpsampleConvLayer
            FinalLayer = LightweightConvLayer
        else:
            UpLayer = UpsampleConvLayer # v1, v2 use standard conv as the decoder
            FinalLayer = ConvLayer

        # ---------------------------------------------------

        # Initial convolution layers (Encoder)
        self.conv1 = EncLayer(3, 32, kernel_size=9, stride=1)
        self.in1 = nn.InstanceNorm2d(32, affine=True)
        self.conv2 = EncLayer(32, 64, kernel_size=3, stride=2)
        self.in2 = nn.InstanceNorm2d(64, affine=True)
        self.conv3 = EncLayer(64, 128, kernel_size=3, stride=2)
        self.in3 = nn.InstanceNorm2d(128, affine=True)

        # Residual layers
        self.res1 = LightweightResidualBlock(128)
        self.res2 = LightweightResidualBlock(128)
        self.res3 = LightweightResidualBlock(128)
        self.res4 = LightweightResidualBlock(128)
        self.res5 = LightweightResidualBlock(128)

        # Upsampling Layers (Decoder)
        self.deconv1 = UpLayer(128, 64, kernel_size=3, stride=1, upsample=2)
        self.in4 = nn.InstanceNorm2d(64, affine=True)
        self.deconv2 = UpLayer(64, 32, kernel_size=3, stride=1, upsample=2)
        self.in5 = nn.InstanceNorm2d(32, affine=True)

        # Final Output Layer
        self.deconv3 = FinalLayer(32, 3, kernel_size=9, stride=1)

        self.relu = nn.ReLU()

    def forward(self, X):
        y = self.relu(self.in1(self.conv1(X)))
        y = self.relu(self.in2(self.conv2(y)))
        y = self.relu(self.in3(self.conv3(y)))
        y = self.res1(y)
        y = self.res2(y)
        y = self.res3(y)
        y = self.res4(y)
        y = self.res5(y)
        y = self.relu(self.in4(self.deconv1(y)))
        y = self.relu(self.in5(self.deconv2(y)))
        y = self.deconv3(y)
        return torch.sigmoid(y)

# ==========================================
# Visualization
# ==========================================

def register_feature_hooks(model, layer_names):
    """
    model: any nn.Module
    layer_names: list of layer names to inspect, e.g., ["conv1", "res1", "deconv1"]
    Returns: feature_maps dict and hooks list
    """
    feature_maps = {}

    def get_hook(name):
        def hook(module, input, output):
            # Save a copy to the CPU for visualization
            feature_maps[name] = output.detach().cpu()
        return hook

    hooks = []
    for name, module in model.named_modules():
        if name in layer_names:
            hooks.append(module.register_forward_hook(get_hook(name)))
    return feature_maps, hooks

# ==========================================
# Verify parameter counts
# ==========================================

def count_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)

baseline = BaselineTransformerNet()
v1 = LightweightTransformerNet("v1")
v2 = LightweightTransformerNet("v2")
v3 = LightweightTransformerNet("v3")

params_baseline = count_parameters(baseline)
params_v1 = count_parameters(v1)
params_v2 = count_parameters(v2)
params_v3 = count_parameters(v3)

print(f"📊 Baseline Model Parameters: {params_baseline:,}")
print(f"🚀 Lightweight-V1 Model Parameters: {params_v1:,}")
print(f"📉 Reduction: {(1 - params_v1 / params_baseline) * 100:.2f}%")
print(f"🚀 Lightweight-V2 Model Parameters: {params_v2:,}")
print(f"📉 Reduction: {(1 - params_v2 / params_baseline) * 100:.2f}%")
print(f"🚀 Lightweight-V3 Model Parameters: {params_v3:,}")
print(f"📉 Reduction: {(1 - params_v3 / params_baseline) * 100:.2f}%")

In [ ]:
def denormalize_img(tensor):
    """
    Input: Tensor (B, C, H, W) or (C, H, W) range [0, 1]
    Output: Numpy Array (H, W, C) range [0, 255]
    """
    img = tensor.clone().detach().cpu()

    if img.dim() == 4:
        img = img[0]

    img = img.permute(1, 2, 0).numpy()

    # Only need to clip for safety; mean/std computation is no longer required
    img = np.clip(img, 0, 1)
    img = (img * 255).astype(np.uint8)

    return img

In [ ]:
def gram_matrix(input):
    batch_size, c, h, w = input.size()
    features = input.view(batch_size, c, h * w)
    G = torch.bmm(features, features.transpose(1, 2))
    return G.div(c * h * w)

class Vgg16Features(nn.Module):
    def __init__(self):
        super(Vgg16Features, self).__init__()
        vgg16 = models.vgg16(weights=VGG16_Weights.IMAGENET1K_V1)
        self.features = vgg16.features.eval()
        for param in self.features.parameters():
            param.requires_grad = False

        # Define the normalization parameters required by VGG
        self.register_buffer('mean', torch.tensor([0.485, 0.456, 0.406]).view(1, 3, 1, 1))
        self.register_buffer('std', torch.tensor([0.229, 0.224, 0.225]).view(1, 3, 1, 1))

    def forward(self, x):
        # Manually normalize before feeding into VGG
        # x is assumed to be in [0, 1]
        x = (x - self.mean) / self.std

        features = []
        style_layers_indices = {3, 8, 15, 22}
        for name, layer in self.features._modules.items():
            x = layer(x)
            if int(name) in style_layers_indices:
                features.append(x)
        return features

class PerceptualLoss(nn.Module):
    def __init__(self, style_img_tensor, content_weight=1e5, style_weight=1e10, tv_weight=0):
        super(PerceptualLoss, self).__init__()
        self.vgg = Vgg16Features().to(device)
        self.content_weight = content_weight
        self.style_weight = style_weight
        self.tv_weight = tv_weight
        self.mse_loss = nn.MSELoss()

        # Compute style Grams (style_img_tensor is now in 0-1; VGG will normalize internally)
        with torch.no_grad():
            style_img_tensor = style_img_tensor.to(device)
            self.style_features = self.vgg(style_img_tensor)
            self.style_grams = [gram_matrix(f).detach() for f in self.style_features]

    def total_variation_loss(self, img):
        bs_img, c_img, h_img, w_img = img.size()
        tv_h = torch.pow(img[:, :, 1:, :] - img[:, :, :-1, :], 2).sum()
        tv_w = torch.pow(img[:, :, :, 1:] - img[:, :, :, :-1], 2).sum()
        return (tv_h + tv_w) / (bs_img * c_img * h_img * w_img)

    def forward(self, generated_img, content_img):
        # generated_img and content_img are both in [0, 1]
        current_batch_size = generated_img.shape[0]

        gen_features = self.vgg(generated_img)
        content_features = self.vgg(content_img)

        # Content Loss (commonly relu2_2, i.e., index 1 in the list)
        content_loss = self.content_weight * self.mse_loss(gen_features[1], content_features[1])

        # Style Loss
        style_loss = 0.0
        for gen_feat, target_gram in zip(gen_features, self.style_grams):
            gen_gram = gram_matrix(gen_feat)
            target_gram_expanded = target_gram.expand(current_batch_size, -1, -1)
            style_loss += self.mse_loss(gen_gram, target_gram_expanded)

        style_loss *= self.style_weight

        # Total Variation Loss
        tv_loss = self.tv_weight * self.total_variation_loss(generated_img)

        total_loss = content_loss + style_loss + tv_loss

        return total_loss, content_loss, style_loss, tv_loss

print("✅ Loss function updated with normalization logic!")

In [ ]:
def train_style_transfer(model, model_name, train_loader, style_img_tensor,
                         fixed_content_image,
                         scheduler_patience,
                         scheduler_factor,
                         val_loader,
                         val_interval,
                         val_batches,
                         epochs=2,
                         lr=1e-3,
                         content_weight=1e5,
                         style_weight=1e10,
                         tv_weight=0,
                         save_dir='./checkpoints'
                         ):

    import os
    os.makedirs(save_dir, exist_ok=True)
    model = model.to(device)

    optimizer = optim.Adam(model.parameters(), lr=lr)

    # Initialize ReduceLROnPlateau scheduler
    # mode='min': triggers when the monitored metric (loss) stops decreasing
    scheduler = ReduceLROnPlateau(optimizer, mode='min',
                                  factor=scheduler_factor,
                                  patience=scheduler_patience
                                  )

    criterion = PerceptualLoss(style_img_tensor, content_weight, style_weight, tv_weight)

    log_dir = os.path.join("runs", model_name)
    writer = SummaryWriter(log_dir=log_dir)
    print(f"📁 TensorBoard log directory: {log_dir}")

    loss_history = []
    start_time = time.time()
    current_step = 0

    print(f"🚀 Starting training: {model_name} | Epochs: {epochs}")

    # Log the original image first
    writer.add_image('images/content', denormalize_img(fixed_content_image), dataformats='HWC')

    model.train()

    for epoch in range(epochs):
        pbar = tqdm(train_loader, desc=f"Epoch {epoch+1}/{epochs}", unit="batch")

        for batch_idx, content_images in enumerate(pbar):
            content_images = content_images.to(device)

            # --- 1. Training step ---
            optimizer.zero_grad()
            generated_images = model(content_images)
            total_loss, c_loss, s_loss, tv_loss = criterion(generated_images, content_images)
            total_loss.backward()
            optimizer.step()

            current_step += 1
            loss_history.append(total_loss.item())

            # Log training loss
            writer.add_scalar('loss/train/total', total_loss.item(), current_step)
            writer.add_scalar('loss/train/content', c_loss.item(), current_step)
            writer.add_scalar('loss/train/style', s_loss.item(), current_step)


            pbar.set_postfix({
                'Loss': f"{total_loss.item():.2e}",
                'LR': f"{optimizer.param_groups[0]['lr']:.2e}"
            })

            # --- 2. Validation step ---
            if val_loader and current_step % val_interval == 0:
                model.eval() # Switch to eval mode

                val_loss_total = 0.0
                val_loss_c = 0.0
                val_loss_s = 0.0
                run_batches = 0

                with torch.no_grad():
                    for i, val_imgs in enumerate(val_loader):
                        if i >= val_batches:
                            break

                        val_imgs = val_imgs.to(device)
                        val_gen = model(val_imgs)
                        v_total, v_c, v_s, _ = criterion(val_gen, val_imgs)

                        val_loss_total += v_total.item()
                        val_loss_c += v_c.item()
                        val_loss_s += v_s.item()

                        run_batches += 1

                if run_batches > 0:
                    avg_v_total = val_loss_total / run_batches
                    avg_v_c = val_loss_c / run_batches
                    avg_v_s = val_loss_s / run_batches

                    writer.add_scalar('loss/val/total', avg_v_total, current_step)
                    writer.add_scalar('loss/val/content', avg_v_c, current_step)
                    writer.add_scalar('loss/val/style', avg_v_s, current_step)

                    # Step the scheduler
                    # Feed the average validation total loss to the scheduler
                    # If that loss fails to drop for 'patience' validations, the LR will be reduced
                    scheduler.step(avg_v_total)
                    # Log the current learning rate
                    current_lr = optimizer.param_groups[0]['lr']
                    writer.add_scalar('train/learning_rate', current_lr, current_step)

                # Visualize images
                val_gen_fixed = model(fixed_content_image)
                vis_gen = denormalize_img(val_gen_fixed)
                writer.add_image('images/generated', vis_gen, current_step, dataformats='HWC')

                # Switch back to training mode
                model.train()

        save_path = f"{save_dir}/{model_name}_{epoch+1}.pth"
        torch.save(model.state_dict(), save_path)

    end_time = time.time()
    duration = end_time - start_time
    save_path = f"{save_dir}/{model_name}.pth"
    torch.save(model.state_dict(), save_path)

    writer.close()
    del criterion
    torch.cuda.empty_cache()

    return loss_history, duration

In [ ]:
# ==========================================
# 2. Run training (Baseline vs. Lightweight)
# ==========================================

# These parameters (epochs, weights) are empirical defaults
EPOCHS = 20
CONTENT_WEIGHT = 1e5
STYLE_WEIGHT = 1e10
TV_WEIGHT = 1e-6  # Add a bit of TV loss to smooth the image
SCHEDULER_PATIENCE=3
SCHEDULER_FACTOR=0.9
VAL_INTERVAL=200
VAL_BATCH=1

print(">>> Sampling a fixed validation image for visualization...")
# Grab one batch from val_loader
val_iter = iter(val_loader)
fixed_val_batch = next(val_iter)
# Take the first image, add batch dim (1, 3, H, W), and move to GPU
fixed_val_image = fixed_val_batch[0].unsqueeze(0).to(device)
print(f"✅ Locked in one validation image, shape: {fixed_val_image.shape}")


print("\n>>> Ready to start the comparison experiments...")

# Launch TensorBoard and point it to the log directory above
!rm -rf ./runs
%load_ext tensorboard
%tensorboard --logdir runs

In [ ]:
# --- Experiment 1: train the Baseline model ---
baseline = BaselineTransformerNet()
loss_history_base, time_base = train_style_transfer(
    baseline,
    "baseline",
    train_loader,
    style_img_tensor,
    fixed_val_image,  # pass in the fixed image
    SCHEDULER_PATIENCE,
    scheduler_factor=SCHEDULER_FACTOR,
    val_loader=val_loader,     # pass in the validation set
    val_interval=VAL_INTERVAL,
    val_batches=VAL_BATCH,
    epochs=EPOCHS,
    content_weight=CONTENT_WEIGHT,
    style_weight=STYLE_WEIGHT,
    tv_weight=TV_WEIGHT
)

In [ ]:
# --- Experiment 2: train the Lightweight - V1 model ---
v1 = LightweightTransformerNet("v1")
loss_history_light, time_light = train_style_transfer(
    v1,
    "lightweight-v1",
    train_loader,
    style_img_tensor,
    fixed_val_image,  # pass in the same fixed image
    scheduler_patience=SCHEDULER_PATIENCE,
    scheduler_factor=SCHEDULER_FACTOR,
    val_loader=val_loader,     # pass in the validation set
    val_interval=VAL_INTERVAL,
    val_batches=VAL_BATCH,
    epochs=EPOCHS,
    content_weight=CONTENT_WEIGHT,
    style_weight=STYLE_WEIGHT,
    tv_weight=TV_WEIGHT
)

In [ ]:
# --- Experiment 3: train the Lightweight - V2 model ---
v2 = LightweightTransformerNet("v2")
loss_history_light, time_light = train_style_transfer(
    v2,
    "lightweight-v2",
    train_loader,
    style_img_tensor,
    fixed_val_image,  # pass in the same fixed image
    scheduler_patience=SCHEDULER_PATIENCE,
    scheduler_factor=SCHEDULER_FACTOR,
    val_loader=val_loader,     # pass in the validation set
    val_interval=VAL_INTERVAL,
    val_batches=VAL_BATCH,
    epochs=EPOCHS,
    content_weight=CONTENT_WEIGHT,
    style_weight=STYLE_WEIGHT,
    tv_weight=TV_WEIGHT
)

In [ ]:
# --- Experiment 4: train the Lightweight - V3 model ---
v3 = LightweightTransformerNet("v3")
loss_history_light, time_light = train_style_transfer(
    v3,
    "lightweight-v3",
    train_loader,
    style_img_tensor,
    fixed_val_image,  # pass in the same fixed image
    scheduler_patience=SCHEDULER_PATIENCE,
    scheduler_factor=SCHEDULER_FACTOR,
    val_loader=val_loader,     # pass in the validation set
    val_interval=VAL_INTERVAL,
    val_batches=VAL_BATCH,
    epochs=EPOCHS,
    content_weight=CONTENT_WEIGHT,
    style_weight=STYLE_WEIGHT,
    tv_weight=TV_WEIGHT
)

In [ ]:
import shutil
import os
import time
from google.colab import files

# Generate a readable timestamp like 2025_01_31_14_23_50
timestamp = time.strftime("%Y_%m_%d_%H_%M_%S")
archive_name = f"{timestamp}.zip"

# Create a temporary directory
temp_dir = "temp_pack"
os.makedirs(temp_dir, exist_ok=True)

# Copy the folders to be packaged
shutil.copytree("checkpoints", os.path.join(temp_dir, "checkpoints"))
shutil.copytree("runs", os.path.join(temp_dir, "runs"))

# Archive
shutil.make_archive(timestamp, "zip", temp_dir)

# Download
files.download(archive_name)

# Clean up
shutil.rmtree(temp_dir)

In [ ]:
import os
import random
import torch
import matplotlib.pyplot as plt
from PIL import Image
import numpy as np
import shutil
from google.colab import files
from torchvision import transforms # Added explicit import if not already in context, but following user prompt assumption

# Assuming device, test_paths, BaselineTransformerNet, LightweightTransformerNet
# and denormalize_img are already defined in the context or previous code blocks

# ==========================
# 0  Set Save Path
# ==========================
output_dir = "test_output"
!rm -rf ./test_output/
os.makedirs(output_dir, exist_ok=True)
print(f"Images will be saved to: {output_dir}")

# ==========================
# 1  Load Models (Keep Unchanged)
# ==========================
ckpt_dir = "./checkpoints"

baseline_eval = BaselineTransformerNet().to(device)
baseline_eval.load_state_dict(
    torch.load(os.path.join(ckpt_dir, "baseline.pth"), map_location=device)
)
baseline_eval.eval()

v1_eval = LightweightTransformerNet("v1").to(device)
v1_eval.load_state_dict(
    torch.load(os.path.join(ckpt_dir, "lightweight-v1.pth"), map_location=device)
)
v1_eval.eval()

v2_eval = LightweightTransformerNet("v2").to(device)
v2_eval.load_state_dict(
    torch.load(os.path.join(ckpt_dir, "lightweight-v2.pth"), map_location=device)
)
v2_eval.eval()

v3_eval = LightweightTransformerNet("v3").to(device)
v3_eval.load_state_dict(
    torch.load(os.path.join(ckpt_dir, "lightweight-v3.pth"), map_location=device)
)
v3_eval.eval()

# ==========================
# 2  Define Image Transforms for Test Set
# ==========================
test_transform = transforms.Compose([
    transforms.Resize(256),
    transforms.CenterCrop(256),
    transforms.ToTensor(),
])

def load_content_image(img_path):
    img_pil = Image.open(img_path).convert("RGB")
    img_tensor = test_transform(img_pil).unsqueeze(0).to(device)
    return img_pil, img_tensor

# ==========================
# 3  Randomly Sample Three Images from Test Set
# ==========================
num_samples = 3
if len(test_paths) < num_samples:
    print(f"Warning: Test set only has {len(test_paths)} images, using all of them.")
    sample_paths = test_paths
else:
    sample_paths = random.sample(test_paths, num_samples)

# ==========================
# 4  Style Transfer: Display + Save
# ==========================
models_for_demo = [
    ("Original", None),
    ("Baseline", baseline_eval),
    ("v1", v1_eval),
    ("v2", v2_eval),
    ("v3", v3_eval),
]

n_rows = len(sample_paths)
n_cols = len(models_for_demo)

plt.figure(figsize=(4 * n_cols, 4 * n_rows))


for row_idx, img_path in enumerate(sample_paths):
    content_pil, content_tensor = load_content_image(img_path)

    # Get bare filename (e.g., "dog.jpg" -> "dog")
    base_name = os.path.splitext(os.path.basename(img_path))[0]

    for col_idx, (name, model) in enumerate(models_for_demo):
        plt_idx = row_idx * n_cols + col_idx + 1
        plt.subplot(n_rows, n_cols, plt_idx)

        image_to_save = None
        save_filename = ""

        if model is None:
            # --- Process Original Image ---
            # Resize original image to 256x256 for consistent display and saving
            vis_transform = transforms.Compose([
               transforms.Resize(256),      # Resize keeping aspect ratio, short edge to 256
                transforms.CenterCrop(256)   # Center crop 256x256
            ])
            resized_pil = vis_transform(content_pil)

            # Matplotlib Display
            plt.imshow(resized_pil)
            plt.title(f"Original\n{base_name}")

            # Prepare for saving
            image_to_save = resized_pil
            save_filename = f"{base_name}.jpg"

        else:
            # --- Process Model Generated Image ---
            with torch.no_grad():
                out_tensor = model(content_tensor)

            # out_img is usually a numpy array (H, W, C)
            out_img = denormalize_img(out_tensor)

            # Matplotlib Display
            plt.imshow(out_img)
            plt.title(name)

            # Prepare for saving: Convert numpy array to PIL Image
            if isinstance(out_img, np.ndarray):
                # Ensure uint8 format for saving
                if out_img.dtype != np.uint8:
                     # Assuming 0-1 range if float; 0-255 if int
                    if out_img.max() <= 1.0:
                        out_img = (out_img * 255).astype(np.uint8)
                    else:
                        out_img = out_img.astype(np.uint8)
                image_to_save = Image.fromarray(out_img)
            else:
                # If denormalize_img returns PIL directly
                image_to_save = out_img

            save_filename = f"{base_name}_{name}.jpg"

        plt.axis("off")

        # --- Execute Save Operation ---
        if image_to_save is not None:
            save_path = os.path.join(output_dir, save_filename)
            image_to_save.save(save_path)
            # Optional: Print save log
            # print(f"Saved: {save_path}")

plt.tight_layout()
plt.show()

print("All images processed and saved to 'test_output' folder.")

# ==========================
# 5  Zip and Download Results
# ==========================

# Set zip archive name (without extension) and directory to zip
zip_name = "test_results"
target_dir = "test_output"  # Corresponds to output_dir defined in previous code

print(f"Zipping '{target_dir}' folder into '{zip_name}.zip' ...")

# 1. Create ZIP archive
# shutil.make_archive(archive_name, format, root_dir)
shutil.make_archive(zip_name, 'zip', target_dir)

# 2. Check if file was generated successfully
zip_file_full_path = f"{zip_name}.zip"
if os.path.exists(zip_file_full_path):
    print(f"Zip successful! File size: {os.path.getsize(zip_file_full_path) / 1024:.2f} KB")

    # 3. Trigger browser download
    print("Starting download, please check your browser prompts...")
    files.download(zip_file_full_path)
else:
    print("Zip failed, please check if folder path exists.")

In [ ]:
input_tensor = fixed_val_image.to("cpu")


def bench(model, name, loops=50):
        model = model.to("cpu")
        model.eval()
        x = input_tensor

        with torch.no_grad():
            for _ in range(5):
                _ = model(x)

        start = time.time()
        with torch.no_grad():
            for _ in range(loops):
                _ = model(x)
        end = time.time()

        t = (end - start) / loops
        print(f"{name} average latency {t*1000:.2f} ms")
        return t




print("=" * 60)
print("⏱️ CPU inference speed comparison")
print("-" * 60)
t0 = bench(baseline, "Baseline")
print("-" * 60)
t1 = bench(v1, "v1")
print(f"🚀 lightweight-v1 speedup {t0/t1:.2f}x")
print("-" * 60)
t2 = bench(v2, "v2")
print(f"🚀 lightweight-v2 speedup {t0/t2:.2f}x")
print("-" * 60)
t3 = bench(v3, "v3")
print(f"🚀 lightweight-v3 speedup {t0/t3:.2f}x")
print("-" * 60)


In [ ]:
# Code Reference Note:
#
# The classes “ConvLayer”, “ResidualBlock”, “UpsampleConvLayer”, and “BaselineTransformerNet” were adapted from
# https://github.com/pytorch/examples/tree/main/fast_neural_style ,
# while the remaining parts were generated by LLM or written by the student.